In [1]:
import re

import pandas as pd
import numpy as np

# 7.1 Handling Missing Data

In [2]:
float_data = pd.Series([1.2, -3.5, np.nan, 0])
float_data

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

In [3]:
# isna() returns a boolean series with True when a value is null
float_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

In [4]:
# None is also treated as na
string_data = pd.Series(["aa", np.nan, None, "hello"])
string_data.isna()

0    False
1     True
2     True
3    False
dtype: bool

There are several helper functions to deal with missing data:

- `dropna`: Filter out rows based on thresholds of how many columns can be empty
- `fillna`: Fill in missing data (method parameter deprecated, use `ffill` or `bfill` now)
- `isna`: Returns a Boolean series indicating which elements are missing
- `notna`: `~isna`

## Filtering Out Missing Data

In [5]:
# dropna drops all rows with missing data
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [6]:
# same effect:
data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [7]:
data = pd.DataFrame([[1.0, 6.5, 3.], [1., np.nan, np.nan], [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [8]:
# on dataframes, a single value in a row is sufficient to be dropped
data.dropna()

,0,1,2
0,1.0,6.5,3.0


In [9]:
# with how="all", only rows where all values are NA are dropped
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [10]:
# dropna also takes in an axis parameter
data[4] = np.nan
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [11]:
data.dropna(axis="columns", how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [12]:
df = pd.DataFrame(np.random.standard_normal((7, 3)))
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
df

,0,1,2
0,0.906009,NaN,NaN
1,0.672730,NaN,NaN
2,0.080677,NaN,-0.727105
3,-0.982163,NaN,2.114522
4,-0.895810,-0.475600,0.311779
5,-1.835040,-0.711155,-1.163336
6,0.950565,-1.334629,-1.843463


In [13]:
# with thresh, you can control more precisely
df.dropna(thresh=2)

,0,1,2
2,0.080677,NaN,-0.727105
3,-0.982163,NaN,2.114522
4,-0.895810,-0.475600,0.311779
5,-1.835040,-0.711155,-1.163336
6,0.950565,-1.334629,-1.843463


## Filling In Missing Data

In [14]:
# fillna takes in a default value and updates all na values
df.fillna(0)

,0,1,2
0,0.906009,0.000000,0.000000
1,0.672730,0.000000,0.000000
2,0.080677,0.000000,-0.727105
3,-0.982163,0.000000,2.114522
4,-0.895810,-0.475600,0.311779
5,-1.835040,-0.711155,-1.163336
6,0.950565,-1.334629,-1.843463


In [15]:
# you can also specify a dictionary, where keys are columns
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,0.906009,0.500000,0.000000
1,0.672730,0.500000,0.000000
2,0.080677,0.500000,-0.727105
3,-0.982163,0.500000,2.114522
4,-0.895810,-0.475600,0.311779
5,-1.835040,-0.711155,-1.163336
6,0.950565,-1.334629,-1.843463


In [16]:
# there are also interpolation methods:
# df.ffill() propagates the last observed value forward
df.bfill()  # propagates the first observed value backwards

,0,1,2
0,0.906009,-0.475600,-0.727105
1,0.672730,-0.475600,-0.727105
2,0.080677,-0.475600,-0.727105
3,-0.982163,-0.475600,2.114522
4,-0.895810,-0.475600,0.311779
5,-1.835040,-0.711155,-1.163336
6,0.950565,-1.334629,-1.843463


In [17]:
# or use another function as input value
df.fillna(df.mean())

,0,1,2
0,0.906009,-0.840461,-0.261520
1,0.672730,-0.840461,-0.261520
2,0.080677,-0.840461,-0.727105
3,-0.982163,-0.840461,2.114522
4,-0.895810,-0.475600,0.311779
5,-1.835040,-0.711155,-1.163336
6,0.950565,-1.334629,-1.843463


# 7.2 Data Transformation

## Removing Duplicates

In [18]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"], "k2": [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [19]:
# duplicated tells us which rows are duplicate
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [20]:
# takes into account all columns
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [21]:
data["v1"] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [22]:
# you can specify which columns to use
data.drop_duplicates(subset=["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [23]:
data.drop_duplicates(subset=["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## Transforming Data Using a Function or Mapping

In [24]:
data = pd.DataFrame(
    {"food": ["bacon", "pulled pork", "bacon", "pastrami", "corned beef", "bacon", "pastrami", "honey ham", "nova lox"],
     "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [25]:
meat_to_animal = {
  "bacon": "pig",
  "pulled pork": "pig",
  "pastrami": "cow",
  "corned beef": "cow",
  "honey ham": "pig",
  "nova lox": "salmon"
}

In [26]:
data["animal"] = data["food"].map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [27]:
# a function works as well
def get_animal(x):
    """Returns animal from meat category"""
    return meat_to_animal[x]
data["food"].map(get_animal)

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

## Replacing Values

In [28]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [29]:
# replace is a simple method for updating values in a dataframe/series
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [30]:
# you can also pass a list
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [31]:
# for both parameters
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [32]:
# or use a dict
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

## Renaming Axis Indexes

In [33]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)), index=["Ohio", "Colorado", "New York"], columns=["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [34]:
def transform(x):
    """Update index name"""
    return x[:4].upper()

data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [35]:
# need to assign
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


## Discretization and Binning

In [36]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
age_categories = pd.cut(ages, bins)
# each element in the return value describes in which range the element is
age_categories

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [37]:
# bin identifier
age_categories.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [38]:
# which categories exists
age_categories.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [39]:
age_categories.value_counts()

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
Name: count, dtype: int64

In [40]:
# control which side of the intervals is open using right=...
pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

In [41]:
# You can also pass in names instead for each interval
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [42]:
# can also pass in an integer instead of bins: pandas will create equally-sized bins
pd.cut(ages, 4, labels=group_names)

['Youth', 'Youth', 'Youth', 'Youth', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [43]:
data = np.random.standard_normal(1000)
# qcut creates bins having the same number of data points
quartiles = pd.qcut(data, 4, precision=2)
quartiles

[(-3.03, -0.59], (-0.59, 0.085], (0.085, 0.77], (-3.03, -0.59], (-0.59, 0.085], ..., (0.085, 0.77], (-3.03, -0.59], (-0.59, 0.085], (-0.59, 0.085], (-0.59, 0.085]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.03, -0.59] < (-0.59, 0.085] < (0.085, 0.77] < (0.77, 3.5]]

In [44]:
quartiles.value_counts()

(-3.03, -0.59]    250
(-0.59, 0.085]    250
(0.085, 0.77]     250
(0.77, 3.5]       250
Name: count, dtype: int64

In [45]:
# can also pass custom quantiles
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]).value_counts()

(-3.025, -1.202]    100
(-1.202, 0.0845]    400
(0.0845, 1.336]     400
(1.336, 3.498]      100
Name: count, dtype: int64

## Detecting and Filtering Outliers

In [46]:
data = pd.DataFrame(np.random.standard_normal((1000, 4)))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.045827,-0.003241,0.002199,0.003816
std,1.032302,0.997560,1.026490,0.985438
min,-3.514391,-2.827117,-3.455131,-3.174049
25%,-0.622100,-0.704674,-0.706526,-0.719145
50%,0.055528,0.000259,0.028407,-0.009312
75%,0.720180,0.702979,0.718420,0.709586
max,3.879800,2.663409,3.269399,2.977735


In [47]:
# find specific values larger than 3
col = data[2]
col[col.abs() > 3]

376   -3.455131
559   -3.192718
858    3.269399
Name: 2, dtype: float64

In [48]:
# if you want any row with a value exceeding 3 in a dataframe
data[(data.abs() > 3).any(axis="columns")]

,0,1,2,3
159,0.178878,0.847409,-0.033450,-3.174049
370,3.266726,1.479944,0.527196,2.606579
376,0.265165,0.160384,-3.455131,0.651407
485,-3.211144,-0.434261,1.884419,0.092500
559,1.631421,-1.091841,-3.192718,0.568873
639,-3.171502,0.562902,0.150985,-0.063483
726,3.067584,0.459846,1.462062,0.062697
857,3.879800,0.460561,-1.273213,1.766039
858,-0.973185,0.928660,3.269399,-0.847756
943,-3.294593,-0.059772,1.867805,0.272679


In [50]:
# can be used e.g. to cap data
data[data.abs() > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.045804,-0.003241,0.002578,0.003990
std,1.024658,0.997560,1.023647,0.984891
min,-3.000000,-2.827117,-3.000000,-3.000000
25%,-0.622100,-0.704674,-0.706526,-0.719145
50%,0.055528,0.000259,0.028407,-0.009312
75%,0.720180,0.702979,0.718420,0.709586
max,3.000000,2.663409,3.000000,2.977735


## Permutation and Random Sampling

In [51]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))
df

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [52]:
sampler = np.random.permutation(5)
sampler

array([1, 4, 0, 3, 2])

In [53]:
# permutate the dataframe
df.iloc[sampler]

,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
4,28,29,30,31,32,33,34
0,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
2,14,15,16,17,18,19,20


In [54]:
# permutate rows
column_sampler = np.random.permutation(7)
column_sampler

array([5, 4, 2, 6, 0, 1, 3])

In [55]:
df.take(column_sampler, axis="columns")

,5,4,2,6,0,1,3
0,5,4,2,6,0,1,3
1,12,11,9,13,7,8,10
2,19,18,16,20,14,15,17
3,26,25,23,27,21,22,24
4,33,32,30,34,28,29,31


In [56]:
# select a random subset without replacement
df.sample(3)

,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
1,7,8,9,10,11,12,13
0,0,1,2,3,4,5,6


In [57]:
# with replacement
df.sample(5, replace=True)

,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
2,14,15,16,17,18,19,20
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20


## Computing Indicator/Dummy Variables

In [58]:
# create one-hot encoding
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"], "data1": range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [59]:
pd.get_dummies(df["key"], dtype=float)

,a,b,c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


In [60]:
# custom column names
dummies = pd.get_dummies(df["key"], prefix="key", dtype=float)
df_with_dummy = df[["data1"]].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0.0,1.0,0.0
1,1,0.0,1.0,0.0
2,2,1.0,0.0,0.0
3,3,0.0,0.0,1.0
4,4,1.0,0.0,0.0
5,5,0.0,1.0,0.0


In [61]:
mnames = ["movie_id", "title", "genres"]
movies = pd.read_table("examples/movies.dat", sep="::", header=None, names=mnames, engine="python")
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [62]:
# special function for multiple categories from strings
dummies = movies["genres"].str.get_dummies("|")
dummies.head()

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [63]:
movies_windic = movies.join(dummies.add_prefix("Genre_"))
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Action                                   0
Genre_Adventure                                0
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Crime                                    0
Genre_Documentary                              0
Genre_Drama                                    0
Genre_Fantasy                                  0
Genre_Film-Noir                                0
Genre_Horror                                   0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Romance                                  0
Genre_Sci-Fi                                   0
Genre_Thriller                                 0
Genre_War                                      0
Genre_Western       

In [64]:
# combine dummies (one-hot encoding) with discretization
np.random.seed(12345)
values = np.random.uniform(size=10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [65]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,False,False,False,False,True
1,False,True,False,False,False
2,True,False,False,False,False
3,False,True,False,False,False
4,False,False,True,False,False
5,False,False,True,False,False
6,False,False,False,False,True
7,False,False,False,True,False
8,False,False,False,True,False
9,False,False,False,True,False


# 7.3 Extension Data Types

In [66]:
# even though the numbers are integer, the type is float, because a missing value is present
s = pd.Series([1, 2, 3, None])
s.dtype

dtype('float64')

In [67]:
# use an explicit extension data type
s = pd.Series([1, 2, 3, None], dtype=pd.Int64Dtype())
print(s.dtype)
s.isna()

Int64


0    False
1    False
2    False
3     True
dtype: bool

In [68]:
# special type for strings: use less memory and are computationally more efficient
s = pd.Series(['one', 'two', None, 'three'], dtype=pd.StringDtype())
s

0      one
1      two
2     <NA>
3    three
dtype: string

In [69]:
# convert extension types
df = pd.DataFrame({"A": [1, 2, None, 4], "B": ["one", "two", "three", None], "C": [False, None, False, True]})
df

,A,B,C
0,1.0,one,False
1,2.0,two,None
2,NaN,three,False
3,4.0,None,True


In [70]:
df["A"] = df["A"].astype("Int64")
df["B"] = df["B"].astype("string")
df["C"] = df["C"].astype("boolean")
df

,A,B,C
0,1,one,False
1,2,two,<NA>
2,<NA>,three,False
3,4,<NA>,True


# 7.4 String Manipulation

## Python Built-In String Object Methods

In [71]:
# split up string
val = "a,b, guido"
val.split(",")

['a', 'b', ' guido']

In [72]:
# get individual parts
pieces = [x.strip() for x in val.split(",")]
pieces

['a', 'b', 'guido']

In [73]:
# join back together
first, second, third = pieces
first + "::" + second + "::" + third

'a::b::guido'

In [74]:
# better pythonic approach
"::".join(pieces)

'a::b::guido'

In [75]:
# find substrings
"guido" in val

True

In [76]:
# check first occurrence
val.index(",")

1

In [77]:
# similar function, but does not raise an error if a substring is not found
val.find(":")

-1

In [78]:
# number of occurrences
val.count(",")

2

In [79]:
# replace parts of a string
val.replace(",", "::")

'a::b:: guido'

In [ ]:
# some common string methods
# count: number of occurrences
# endswith: check whether string ends with some suffix
# startswith: check whether string starts with some prefix
# join: concatenate some sequence of other strings
# index: find first occurrence
# find: same, but no error is thrown if the substring does not exist
# rfind: find last occurrence
# replace: replace substrings with new sequence
# strip, rstrip, lstrip, trim whitespace on both, left or right sides
# split: break up string into a list of strings
# lower: convert to lowercase
# upper: convert to uppercase
# ...

## Regular Expressions

- a describes some pattern to locate in text
- can be used for pattern matching, substitution and splitting

In [80]:
text = "foo    bar\t baz  \tqux"
re.split(r"\s+", text)

['foo', 'bar', 'baz', 'qux']

In [81]:
# regex is first compiled, you can compile yourself too
regex = re.compile(r"\s+")
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [82]:
# get a list of all matching patterns
regex.findall(text)

['    ', '\t ', '  \t']

In [83]:
# search only returns the first match
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com"""
pattern = r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}"

regex = re.compile(pattern, flags=re.IGNORECASE)
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [84]:
# search only returns the first occurrence as a special object
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [85]:
text[m.start():m.end()]

'dave@google.com'

In [86]:
# regex.match matches only the start of the string
regex.match(text)   # no output, because the string does not start with a match

In [87]:
# replace
regex.sub("REDACTED", text)

'Dave REDACTED\nSteve REDACTED\nRob REDACTED\nRyan REDACTED'

In [88]:
# find a pattern and segment it based on a regex
# put parentheses into the pattern
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
regex = re.compile(pattern, flags=re.IGNORECASE)

m = regex.match("wesm@bright.net")
m.groups()

('wesm', 'bright', 'net')

In [89]:
# this can also be done with several matching groups
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [90]:
# can also replace based on the groups using the special \<number> format
print(regex.sub(r"Username: \1, Domain: \2, Suffix: \3", text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com


**Summary**:

- `findall`: Return all non-overlapping matching patterns in a string as a list
- `finditer`: Like findall, but returns an iterator
- `match`: Match patterns at start of string and optionally segment pattern components into groups
- `search`: Scan a string for matching pattern (anywhere in the string)
- `split`: Break a string into pieces
- `sub` and `subn`: Replace all or just the first `n` occurrences of a pattern

## String Functions in pandas

In [91]:
data = {"Dave": "dave@google.com", "Steve": "steve@gmail.com", "Rob": "rob@gmail.com", "Wes": np.nan}
data = pd.Series(data)
data.name = "Email"

In [92]:
data.isna()

Dave     False
Steve    False
Rob      False
Wes       True
Name: Email, dtype: bool

In [93]:
# map can be used to apply functions to each value, but they fail on missing values
# Series has array-oriented methods for string operations as well, to better cope with this
data.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes        NaN
Name: Email, dtype: object

In [94]:
# also possible with extension types
data_as_string_ext = data.astype("string")
data_as_string_ext.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes       <NA>
Name: Email, dtype: boolean

In [95]:
# regex can also be used, along any flags
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
Name: Email, dtype: object

In [96]:
# can now do vectorized element retrieval
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]
matches

Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
Name: Email, dtype: object

In [97]:
matches.str.get(1)

Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
Name: Email, dtype: object

In [98]:
# or slice strings
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
Name: Email, dtype: object

In [99]:
# extract will create a whole dataframe
pattern = r"(?P<Name>[A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"   # ?P<name> to define column name
data.str.extract(pattern, flags=re.IGNORECASE)

,Name,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


# 7.5 Categorical Data

## Background and Motivation

In [100]:
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [101]:
# already know these methods
values.unique()

array(['apple', 'orange'], dtype=object)

In [102]:
values.value_counts()

apple     6
orange    2
Name: count, dtype: int64

In [103]:
# typical approach in data systems for more efficient storage and computation:
# use dimension tables: store integer keys explicitly
values = pd.Series([0, 1, 0, 0] * 2)
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [104]:
# and a dimension table mapping key to value
dim = pd.Series(['apple', 'orange'])
dim

0     apple
1    orange
dtype: object

In [105]:
# now use take to restore original
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

## Categorical Extension Type in pandas

In [106]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
n = len(fruits)

rng = np.random.default_rng(seed=123456)
df = pd.DataFrame({'fruit': fruits, 'basket_id': np.arange(n),
                   'count': rng.integers(3, 15, size=n), 'weight': rng.uniform(0, 4, size=n)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,4,3.624204
1,1,orange,10,1.827878
2,2,apple,12,1.122134
3,3,apple,7,2.544595
4,4,apple,7,3.922843
5,5,orange,3,3.249727
6,6,apple,3,1.220793
7,7,apple,14,2.272044


In [107]:
# pandas has a special extension type for categorical data
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [108]:
c = fruit_cat.array
type(c)

pandas.core.arrays.categorical.Categorical

In [109]:
# different categories
c.categories

Index(['apple', 'orange'], dtype='object')

In [110]:
# and codes
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [37]:
# get mapping from categories to codes:
# noinspection PyTypeChecker
dict(enumerate(c.categories))

{0: 'apple', 1: 'orange'}

In [111]:
# change type
df["fruit"] = df["fruit"].astype("category")

In [112]:
# create directly from other python sequence
my_categories = pd.Categorical(["foo", "bar", "baz", "foo", "bar"])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [113]:
# can also create categorical data if only the encoding structure is known
categories = pd.Index(["foo", "bar", "baz"])
codes = [0, 1, 2, 0, 0, 1]
cats_2 = pd.Categorical.from_codes(codes, categories)
cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [114]:
# there is no default ordering assumed, but it can be enforced explicitly
ordered_cats = pd.Categorical.from_codes(codes, categories, ordered=True)
ordered_cats

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [115]:
# can also be enforced later
cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

## Computations with Categoricals

In [116]:
rng = np.random.default_rng(seed=12345)
draws = rng.standard_normal(1000)
draws[:5]

array([-1.42382504,  1.26372846, -0.87066174, -0.25917323, -0.07534331])

In [117]:
# Create 4 equally sized bins based on quantiles
bins = pd.qcut(draws, 4)
bins

[(-3.121, -0.675], (0.687, 3.211], (-3.121, -0.675], (-0.675, 0.0134], (-0.675, 0.0134], ..., (0.0134, 0.687], (0.0134, 0.687], (-0.675, 0.0134], (0.0134, 0.687], (-0.675, 0.0134]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.121, -0.675] < (-0.675, 0.0134] < (0.0134, 0.687] < (0.687, 3.211]]

In [118]:
# Rename quartiles
bins = pd.qcut(draws, 4, labels=["Q1", "Q2", "Q3", "Q4"])
bins

['Q1', 'Q4', 'Q1', 'Q2', 'Q2', ..., 'Q3', 'Q3', 'Q2', 'Q3', 'Q2']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [119]:
bins.codes[:10]

array([0, 3, 0, 1, 1, 0, 0, 2, 2, 0], dtype=int8)

In [120]:
# We can use groupby() to get summary statistics
bins = pd.Series(bins, name="quartile")
results = (pd.Series(draws).
           groupby(bins, observed=False)
           .agg(["count", "min", "max"])
           .reset_index())
results

,quartile,count,min,max
0,Q1,250,-3.119609,-0.678494
1,Q2,250,-0.673305,0.008009
2,Q3,250,0.018753,0.686183
3,Q4,250,0.688282,3.211418


In [121]:
d = pd.Series(draws).groupby(bins, observed=True)
d.describe()

,count,mean,std,min,25%,50%,75%,max
quartile,,,,,,,,
Q1,250.0,-1.294347,0.511478,-3.119609,-1.580714,-1.157943,-0.885171,-0.678494
Q2,250.0,-0.303875,0.203074,-0.673305,-0.458369,-0.290035,-0.120900,0.008009
Q3,250.0,0.343512,0.191344,0.018753,0.193462,0.335801,0.518895,0.686183
Q4,250.0,1.287744,0.491929,0.688282,0.902379,1.187406,1.555319,3.211418


In [122]:
# categoricals can improve performance
n = 10_000_000
labels = pd.Series(["foo", "bar", "baz", "qux"] * (n // 4))
categories = labels.astype("category")

In [123]:
labels.memory_usage(deep=True)

520000132

In [124]:
# categoricals use much less memory
categories.memory_usage(deep=True)

10000512

In [125]:
# Conversion takes some time, though
%time _ = labels.astype("category")

CPU times: user 171 ms, sys: 12 ms, total: 183 ms
Wall time: 184 ms


In [126]:
%timeit labels.value_counts()

200 ms ± 4.01 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [127]:
# However, operations on categoricals are faster, such as groupby() or value_counts()!
%timeit categories.value_counts()

12.3 ms ± 55.6 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Categorical Methods

There are special methods for categorical data.

In [128]:
s = pd.Series(["a", "b", "c", "d"] * 2)
cat_s = s.astype("category")
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [129]:
# with the accessor `cat`, categorical methods can be accessed

# gives us the codes (a number identifier for each category is created)
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [130]:
# all observed categories
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [131]:
# if some categories are observable, but have not been observed, yet
# we can define them!
actual_categories = ["a", "b", "c", "d", "e"]
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2.cat.categories

Index(['a', 'b', 'c', 'd', 'e'], dtype='object')

In [132]:
# the data is still unchanged, but methods account for these categories now
cat_s2.value_counts()

a    2
b    2
c    2
d    2
e    0
Name: count, dtype: int64

In [133]:
cat_s3 = cat_s[cat_s.isin(["a", "b"])]
cat_s3.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [134]:
# you can remove unused categories
cat_s3 = cat_s3.cat.remove_unused_categories()
cat_s3.cat.categories

Index(['a', 'b'], dtype='object')

In [135]:
# other methods
c4 = cat_s3.cat.add_categories(["d"])
c4.cat.categories

Index(['a', 'b', 'd'], dtype='object')

In [136]:
c = pd.Series(["a", "b", "c", "a"], dtype="category")
c = c.cat.as_ordered()
c.cat.categories

Index(['a', 'b', 'c'], dtype='object')

In [138]:
# marks categories as ordered
c6 = c.cat.as_ordered()
c6.cat.categories

# as_unordered() exists as well

Index(['a', 'b', 'c'], dtype='object')

In [139]:
# create one-hot encodings
cat_s = pd.Series(["a", "b", "c", "d"] * 2, dtype="category")
pd.get_dummies(cat_s, dtype=float)

,a,b,c,d
0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0
6,0.0,0.0,1.0,0.0
7,0.0,0.0,0.0,1.0
